In [1]:
# LLM Agents in Google ADK: Packing List & Itinerary Example
# Welcome 👋🏻 In this notebook, you'll install Google’s Agent Development Kit (ADK), 
# configure your environment, define two LLM agents, and run your agent interactions—all in just a few steps.
# You'll see how LLM Agents power real-world travel apps like WanderWise, using natural language understanding, 
# reasoning, and tool integration to deliver smart, actionable results.

In [2]:
# load API Key set in .EnvironmentError
import dotenv
dotenv.load_dotenv()

import os
#import getpass

# Load the Gemini API key from environment variable
# If not set, prompt the user to enter it
api_key = os.environ.get("GEMINI_API_KEY")
if not api_key:
    print("GEMINI_API_KEY not found in environment variables.")
    #api_key = getpass.getpass("Enter your GEMINI_API_KEY: ")
    #os.environ["GEMINI_API_KEY"] = api_key

print("API key configured successfully." if api_key else "Failed to set API key.")

API key configured successfully.


In [3]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

Name: google-adk
Version: 1.18.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: /usr/local/python/3.12.1/lib/python3.12/site-packages
Requires: anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, typing-extensions, tzlocal, uvicorn, watchdog, websockets
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Configure environment
import os

# Set GOOGLE_GENAI_USE_VERTEXAI to "False" to use the public Gemini API directly,
# rather than routing through Google Cloud's Vertex AI. This simplifies setup for quick demos.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Define the specific Gemini model we'll use. 
# 'gemini-2.0-flash' is a fast and efficient model.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [6]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

# InMemorySessionService: A simple, in-memory service for managing conversation sessions. 
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# types from google.genai: Provides data structures (like Content and Part) to represent messages exchanged with the language model.
from google.genai import types

from IPython.display import Markdown, display

In [7]:
# Modular Function for Agent Interaction
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> str:
    """
    Runs a single interaction with an ADK agent and returns its final text response.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        The final text response from the agent as a string. Returns
        "No final response from agent." if no final response event is found.
    """
    # Use the provided session service if available, otherwise create a new one.
    # This flexibility allows for both isolated examples and continuous conversations.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session. In a multi-turn conversation, you would reuse
    # the same session_id across turns to maintain context. 
    # For these distinct examples, we'll use unique session_ids.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object, which is how messages
    # are structured for the agent.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner: This is the engine that connects the user's message
    # to the agent and manages the session state.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent: This sends the user's message to the agent for processing.
    # The 'runner.run()' method returns an iterable of events that describe
    # the agent's actions and responses.
    events = list(runner.run(user_id=user_id, session_id=session_id, new_message=content))
    # Copilot added the list(), before it was:
    #  events = runner.run(user_id=user_id, session_id=session_id, new_message=content)
    # See Copilot fix notes at the bottom for more info
    
    # ** Suggested by Copilot to print the raw event and each part to inspect available fields **
    # Print the raw event and each part to inspect available fields:
    if False:  # Set to True to enable debug printing
        for event in events:
            print("EVENT:", event)
            for i, part in enumerate(event.content.parts):
                print(" PART", i, "repr:", repr(part))
                # Show common possible attributes
                for attr in ("text", "html", "url", "uri", "href", "link"):
                    if hasattr(part, attr):
                        print("  ", attr, "=", getattr(part, attr))

    # This is the new version created by Copilot to handle the links in grounding metadata
    # before it was:
    #     # Process the events to find and extract the final response from the agent.      
    #     for event in events:
    #         if event.is_final_response():
    #             # Concatenate all parts in the final response
    #             response = ''.join(part.text for part in event.content.parts if part.text)
    #             return response

    # Helper to get readable text for a Part
    def _part_to_text(part):
        if getattr(part, "text", None):
            return part.text
        if getattr(part, "html", None):
            return part.html
        for attr in ("url", "uri", "href", "link"):
            val = getattr(part, attr, None)
            if val:
                return f"[{val}]({val})"
        return str(part)

    # Build final response and include grounding links if available
    for event in events:
        if event.is_final_response():
            # Combine the parts' text/HTML
            response = "".join(_part_to_text(p) for p in event.content.parts)

            # Try to append grounding sources (if present)
            sources = []
            gm = getattr(event, "grounding_metadata", None)
            if gm and getattr(gm, "grounding_chunks", None):
                for chunk in gm.grounding_chunks:
                    web = getattr(chunk, "web", None)
                    if web:
                        title = getattr(web, "title", None) or getattr(web, "uri", None)
                        uri = getattr(web, "uri", None)
                        if uri:
                            # Use the title if present, otherwise display the URL as the label
                            label = title if title else uri
                            sources.append(f"- [{label}]({uri})")

            if sources:
                response += "\n\n**Sources:**\n" + "\n".join(sources)

            return response
    return "No final response from agent."

print("Modular function `run_adk_agent_interaction` successfully defined.")

# # NOTE: More details about Runners, Sessions, and advanced event processing are coming up in later tutorials—stay tuned!

Modular function `run_adk_agent_interaction` successfully defined.


In [8]:
# Define the Packing List Agent
# Our first agent, the packing_list_agent, is designed to generate a comprehensive packing list based on a user's travel details. 
# The key here is its instruction set, which guides the agent to ask clarifying questions and organize its output in a clear, categorized Markdown format.
# name: A unique identifier for the agent.
# model: The specific Gemini model (gemini-2.0-flash) powering the agent's reasoning.
# description: A brief summary of the agent's function.
# instruction: The detailed prompt that defines the agent's behavior, including what information to seek and how to format the output.
packing_list_agent = Agent(
    name="packing_list_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that generates a packing list for a trip given a destination, duration, and user preferences.",
    instruction="""
        You are a helpful travel assistant.
        When the user provides a destination and trip duration, generate a detailed packing list tailored for that location and timeframe.
        Ask about planned activities and any special preferences or needs (such as business, hiking, or traveling with children).
        If the user wishes, you may ask about personal preferences (such as specific clothing or toiletries), but do not assume based on gender.
        Make sure your suggestions are practical and organized by category (clothing, toiletries, electronics, documents, etc.).
        If the user provides additional details, refine the list accordingly.
        Present the packing list in a clear and easy-to-read markdown format.
    """,
)
# NOTE: In future lessons, you will integrate external tools (e.g., weather APIs) for even more personalized lists.
print("`packing_list_agent` successfully defined.")

`packing_list_agent` successfully defined.


In [9]:
#  Define common parameters for our interactions
APP_NAME = "wanderwise_app"
USER_ID = "user_001"

# Create a common session service instance to reuse.
common_session_service = InMemorySessionService()

# Define the user's input for the packing list agent.
PACKING_LIST_USER_INPUT = "I’m a 21 yo female traveling to Banff for 5 days in July. I plan to do some easy hiking and some city sightseeing."
PACKING_LIST_SESSION_ID = f"{APP_NAME}_packing_list_session_001" # Unique session ID for this interaction

print("--- Running Packing List Agent ---")

# Invoke the packing list agent using our modular helper function.
packing_list_response = await run_adk_agent_interaction(
    agent=packing_list_agent,
    user_id=USER_ID,
    session_id=PACKING_LIST_SESSION_ID,
    input_text=PACKING_LIST_USER_INPUT,
    app_name=APP_NAME,
    session_service=common_session_service
)

# Display the agent's response, rendering the Markdown for a nicely formatted list.
display(Markdown(f"{packing_list_response}"))

print("--- Packing List Agent Interaction Complete ---")

--- Running Packing List Agent ---


Okay, I can help you generate a packing list for your 5-day trip to Banff in July! Here's a detailed packing list tailored for easy hiking and sightseeing, keeping in mind the time of year:

**Clothing:**

*   **Tops:**
    *   5-7 Short-sleeved shirts (moisture-wicking if possible, good for hiking)
    *   1-2 Long-sleeved shirts (for layering and sun protection)
    *   1 Light fleece or sweater
    *   1 Waterproof/windproof jacket
*   **Bottoms:**
    *   1-2 pairs of hiking pants or convertible pants
    *   1 pair of jeans or casual pants
    *   1 pair of shorts or hiking skirt
*   **Undergarments:**
    *   5-7 pairs of underwear
    *   5-7 pairs of socks (wool or synthetic hiking socks recommended)
    *   1-2 bras (sports bras recommended for hiking)
*   **Sleepwear:**
    *   Pajamas or comfortable sleepwear
*   **Swimwear:**
    *   Bathing suit (if you plan to swim in a lake or hot springs)
*   **Accessories:**
    *   Hat (for sun protection)
    *   Sunglasses
    *   Bandana or Buff (versatile for sun, dust, or warmth)

**Shoes:**

*   Hiking shoes or boots (broken-in and comfortable)
*   Comfortable walking shoes or sneakers
*   Sandals or flip-flops (optional, for relaxing)

**Toiletries:**

*   Sunscreen (high SPF)
*   Insect repellent
*   Lip balm with SPF
*   Shampoo, conditioner, and body wash
*   Toothbrush, toothpaste, and floss
*   Deodorant
*   Face wash and moisturizer
*   Any personal hygiene items
*   Hand sanitizer
*   Small first-aid kit (band-aids, pain relievers, blister treatment)

**Electronics:**

*   Phone and charger
*   Camera and charger (optional)
*   Portable power bank (optional)
*   Adapter (if needed)

**Documents:**

*   Driver's license or government-issued ID
*   Credit cards and cash
*   Health insurance card
*   Reservations and confirmations (hotel, tours, etc.)

**Other:**

*   Backpack (for day hikes)
*   Water bottle or hydration reservoir
*   Snacks (for hiking)
*   Small binoculars (optional, for wildlife viewing)
*   Reusable shopping bag
*   Headlamp or flashlight
*   Bear spray (available for purchase in Banff, know how to use it)
*   Book or entertainment for downtime

**Additional Considerations:**

*   **Weather:** July in Banff can have variable weather. Be prepared for sunshine, rain, and cooler temperatures, especially at higher elevations.
*   **Hiking:** Bring layers, even on seemingly warm days, as conditions can change quickly in the mountains.
*   **Bugs:** Mosquitoes and other insects can be prevalent, especially near water.

Do you have any specific preferences or needs that I should add to this list? For example, are there any specific brands of clothing or toiletries you prefer, or any other activities you might be interested in?


--- Packing List Agent Interaction Complete ---


In [10]:
# Congratulations 🎉  You’ve just defined and successfully run your first LLM agent for a real-world task: generating a personalized packing list. 
# This demonstrates how easily you can instruct an LLM to perform structured tasks and manage user interactions within the ADK framework.

In [11]:
# Itinerary Agent with Google Search Tool
# Now, let's go a step further: create an Itinerary Agent that uses the google_search tool! 
# This agent will be capable of generating a personalized itinerary for your trip, 
# leveraging live search results to find up-to-date recommendations for attractions, restaurants, and activities.

In [12]:
# Import the Google Search tool from ADK.
# This tool provides the agent with the ability to perform web searches.
from google.adk.tools import google_search

print("`google_search` tool imported.")

`google_search` tool imported.


In [13]:
# Define the Itinerary Agent
# The agent's instruction should explicitly guide it on when and how to use these tools. 
# Notice the instruction use the [google_search] tool. 
# This syntax helps the LLM understand that it has access to a tool named google_search and encourages it to use it appropriately.
itinerary_agent = Agent(
    name="itinerary_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that creates a travel itinerary for a given destination and trip duration, using Google Search for up-to-date recommendations.",
    instruction="""
        You are a helpful and creative travel itinerary planner.
        When the user provides a destination and trip duration, use the [google_search] tool to find current and popular attractions, restaurants, and activities for each day.
        For each day, create a detailed schedule with 2-4 activities, including at least one meal suggestion and one local attraction, prioritizing the user's interests (such as art and food).
        Present the itinerary in a clear, easy-to-read markdown format, organized by day.

        Example format:

        ### Day 1
        - Morning: [Attraction or activity]
        - Lunch: [Restaurant or food experience]
        - Afternoon: [Attraction or activity]
        - Evening: [Dinner suggestion or event]

        ### Day 2
        ...

        Always use the [google_search] tool to find the latest recommendations for each activity or restaurant.
        When providing recommendations, if a direct and relevant official website or information page URL is prominently found in the search results for a specific item, include it as a Markdown hyperlink like [Item Name](URL).
        Be concise, friendly, and ensure the itinerary is complete for all days requested.
        Don't forget to include the links in the answer as a Markdown hyperlink like [Item Name](URL).
    """,
    tools=[google_search], # This is the crucial part: assigning the tool to the agent!
)

print("`itinerary_agent` successfully defined with `google_search` tool.")

`itinerary_agent` successfully defined with `google_search` tool.


In [14]:
# Run the Itinerary Agent
ITINERARY_USER_INPUT = "I'm visiting Barcelona for 3 days and I love art, food, and dance. Can you plan my trip?"
ITINERARY_SESSION_ID = f"{APP_NAME}_itinerary_session_001" # Unique session ID for this interaction

print("--- Running Itinerary Agent with Google Search ---")

# Invoke the itinerary agent using our modular helper function.
# This agent will internally decide to use the `google_search` tool based on its instructions.
itinerary_response = await run_adk_agent_interaction(
    agent=itinerary_agent,
    user_id=USER_ID,
    session_id=ITINERARY_SESSION_ID,
    input_text=ITINERARY_USER_INPUT,
    app_name=APP_NAME,
    session_service=common_session_service
)

# Display the agent's response, rendering the Markdown.
display(Markdown(f"{itinerary_response}"))

print("--- Itinerary Agent Interaction Complete ---")

--- Running Itinerary Agent with Google Search ---


Okay! Here is a possible itinerary for your 3-day trip to Barcelona, focusing on art, food, and dance.

Here is an itinerary for your 3 days in Barcelona:

### Day 1: Art and Gothic Charm

*   **Morning:** Start your day at the [Picasso Museum](https://www.museupicasso.bcn.cat/), which focuses on Pablo Picasso's early years and his relationship with Barcelona.
*   **Lunch:** Enjoy traditional Catalan tapas at [Bar del Pla](https://www.barcelona-tapas.com/bar-del-pla.html) in the El Born neighborhood.
*   **Afternoon:** Wander through the Gothic Quarter, exploring the [Barcelona Cathedral](https://www.catedralbcn.org/ and the Plaça Reial.
*   **Evening:** Experience a passionate [flamenco show at Tablao Cordobes](https://www.tablaocordobes.com/en/), known for its authentic performances and dining options.

### Day 2: Gaudí's Masterpieces and Culinary Delights

*   **Morning:** Visit [Park Güell](https://parkguell.barcelona/en), a whimsical park designed by Antoni Gaudí. Pre-book your tickets online to ensure entry.
*   **Lunch:** Have lunch at [El Quim de la Boqueria](https://www.timeout.com/barcelona/restaurants/el-quim-de-la-boqueria), a tapas bar located within La Boqueria Market, offering a wide array of Catalan dishes.
*   **Afternoon:** Explore [Casa Batlló](https://www.casabatllo.es/en/), one of Gaudí's most famous buildings. Consider purchasing a skip-the-line ticket in advance.
*   **Evening:** Indulge in a memorable dining experience at [Disfrutar](https://www.disfrutarbarcelona.com/en/), a two-Michelin-starred restaurant known for its innovative and avant-garde cuisine.

### Day 3: Modern Art and Musical Inspiration

*   **Morning:** Visit the [Museu Nacional d'Art de Catalunya (MNAC)](https://www.museunacional.cat/en), which features a comprehensive collection of Catalan art from the Romanesque period to the mid-20th century.
*   **Lunch:** Enjoy tapas at [Casa Rincon de Gracia](https://casarincon.com/), known for its authentic tapas.
*   **Afternoon:** Take a guided tour of the [Palau de la Música Catalana](https://www.palaumusica.cat/en/), a concert hall designed in the Catalan modernism style.
*   **Evening:** Enjoy a final taste of Barcelona with a [tapas tour in the El Born neighborhood](https://www.eyeonfoodtours.com/), sampling local favorites.


**Sources:**
- [wonderfulmuseums.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEIspPfTccwNNZaVN0bBmQAEqOIW62ibh8P6Y0qwIKB-MZbusb2fsC6OJ3KEsVSZVKAhMXHRak0xN5P23p2W-cMT3wM9KIXqRZHETD2rF9u3s1z7tgwlMtH6rJqgXYZksRcC8xylCM4cFRlBZaRSCyReP1Q-oGfJjFuY-e5csUyWhI=)
- [thepostcard.me](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHSqo6T-BL921KuIUXVCukW_gj-4k6LGpvZMHSZqvWbRUTObHVeEmLCoU44lO846gt1i8cgVwrJqNgH_FdfNDuYE9Yr7tbBHr3gqTmDmEQisc5F4zTpvxTSIyjiggGr2khQYc2A1h0NdQztLe8=)
- [eyeonfoodtours.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFrxhHMLRcwL2vEei2MzSu8VOvYbgn5zieV_Jcnx_jrpnjo1M9tqah2tRt8BJSSaXT5F4TYWJ1WEtcSABwenOWYkQSoYo9_7Wyr7PuzOc6375FkpnbGHdcpXc9BW9Q8hb-53HppEtEdb3tPF2T9NhS8yEavp8OElN3ooI6WQ4QwGsCnJRmkBTakYnYoPMUFao-jN5eABpan8qLo_gMV)
- [the500hiddensecrets.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGNc3YFebCzU8IPB4yCZZiPTTIAAobgU6TOU0OP_Gbt3n7ZI0GKWkJYJTayl0KNSFb5VFjkohUTjShuDMFaphprUu85tM-gtENi3bdiro99E_hWCBjfbSm8AuFVi-Z1Ao8Es3KzuzZORcyh5TTvDUtvGh_hzw3ULBoZvZFDQ33oBcu1)
- [emmaplunkett.art](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHmPFKUW-hFPrt5bKGzUKkje7qZGaH1g1Z-jYHtQlyD4bopXm-RSbtu695F0HPGq10BJmItCvJG5cXgOS1sN2fXdKpbDioswJ3uGKKuHx55OiP98YYsHBqyTZjGFVx3CoNU-EC0Yg93x9452v5muORgAywRcqiodkXqMoIc3Q==)
- [barcelonahacks.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEilIhmIUzATekSnoZbNJ9RYUqPutMDQpowpSRD6Xkk_DXkHVK83-S528XYWeBN5Zk0_kF6BDNwpxzrUWWNK-xsJAyQWbySX-rmTHEX4qLzvJZadG4l9nBpuaYuo1U70_0vTOsz84EIBc9dHfmFDn69Lg==)
- [splendidlyspain.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGjE5xEyEELPbv5ey6zjqP-WH3Aj_m3uxuLz-lStsFryJirqkKLHdALbSGOa1oc07QAeU-8k_7F3d61CIKe9NTXzOS7ttzwWs24QUDq4tBf5LCn7o6Cv3axlqIrYXiu3JeQkXG7BVqD6fQKNDucaIVfWg==)
- [bonjourbarcelone.fr](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEFkCMfbNIj1Ng1jcqr_YukaRvFq5Cnu1WsEldkCjbo1VsuZkcKBi5LDEyIfPW8jp8huUW8gQId9GtA9hewfVznGQf7dsvYDox73xYoehwptEZlLIedD9Y_EoawJEb8ZjZzAcJIsOD3mbaKxyLBnGU=)
- [barcelonahacks.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHzhuEoHSVJ9VLrO01Iy8WKmkTJIKwv4hYlqOHSfjz4DcX9WyaNh1aYgxMjW6MEV1Bh9_y429eD_PXmctn95D2u2_-1mFQGCI7GMMK9rIvpFljpJuHYC7iaOdZpZfkmKXKhB9HLvSuV3MadFQ==)
- [klook.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHPgrEZsm4c8rgUk7bQ13FAnHwzSB2_MlHOuwnPb1cV8ccwa0OwE5A-h_P-ukbxw1AqDQo6gpUNzfUvn_sFogO0hAfCBxChfwuyZkYLc1IMNipO3XxiIs-WeLTPkDXPmpekGtPAAdODT-hFTDM47TDBJghPiv6U-H_Nl0RcjKWVmBw8Wg==)
- [barcelona-life.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHTNdjkIOutVqrLG3OyBEXxG78zJsf-K5dAuk9ydvtf1oW5SsbFT-guezWyjA5kWawQ7-7mOLNUhgzenNfCVsOiVBZWEjT3PQogwcCQd-6PAptQzaZ0mSrh9RRmfKUJ9VEuOz5berAkXKdSEmc=)
- [flamenco-show-tickets.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGJhNimH4aXoFe4uNFIxWt_VIP3Nk5bOuxiAIsgo-vWBKmMUhTvQ0bF-wM1eRAQc7gTrVCVS3vRKNDbu4y6_uxEQmHyEB4ZDISP60_9HV8VtzY3iPTTEKbvCrawEucigpO-l26TEipkjO0eHw==)
- [viator.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGJGuspmkgsdUx8KyI-fwRCtreLO5o5jakF6JFSb6GmEXjo6LFMCsQS2-m6k36NEnF6DeBHiXcN-WkkAIqwKfTUbG3QPCKHoRwz6zCfWHETJx6WwRkivcXhRzNDfZt39RUrMqehYyr_KydfsWrYHEgqacyOafmrvaeZEg==)
- [tiqets.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHMIECCiHSrEfi0Kq8m9t65SOYNZvN9p2grjb2PMI9UQqbAgHkS0WNB96Npt8aYI7wyVHVsPnzTflgWIYESSpR8avgFpdmPfDehs6emO_ynLpMEmq_jB3Ftf6HE3j8ZRgzLDbJuDP1iM1BqG63kDRzc)
- [park-guell.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE4pxzugOtYxr3t042tanmjTSYcJ0VmvmSBOu0gpu3S4unbhUtM2jRyOfH5Ow4y6gVnsmqs9bselPdn5jrmSfcpBUxSedd9TSxcrm4ZJ4dD_qh-)
- [viator.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFifouMWW-qHdCCySmb8_spjntwu_pJxa-M9p8a6X9BJgV92Vo3AyABgO6uPJI-HVuankVItWF0th-BZ_du2KdRB0CCnSmxgIDvLrdbSD7Skt2nBmqcXS8gr1CEmeiVHT8D1tyDFWXWzYIaLaGJTcFkVY22PYTrll9jc0PN)
- [parkguell.barcelona](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUx8KyrjQx4fKK0suWYaw-aY5pgtDJlOVS1B1GtZuEyVNmazgj2tKgPbviOdrM-Hm7altjSSpf69ONKIM0mnybS4QFe8nbJ2bibHsUD2LmtfAtG9KM1RqNSg==)
- [barcelona.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGQYzm4iNPbVVHjApkGZodJrWM68qAY7pHNVzwrqZ0g8z8lJLpOx-Zv8ctcJeFYDntEsOko79Xc_lrsGr1JJyqNFcdzizfAgCIV0dRGVR3YpqOdCgRFzEq_6wf7C8HNblzEJlHlExUKy8YVl6IwsK68Bhg74siZuaE=)
- [onlybyland.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGQV2juDvnVRURzPblRfWBIjd-8bSwcIf3BzIHxc1ZXrjwzQCD0415FKaRYH-DqRHZft7jPNP7KpseXjp9vGRuq8ZmjA8Y-Uw78J4AsaQQpiKxk_njCvpBHiSJdDqvrIoY6guQw-e69vx7I)
- [barcelonayellow.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHg2KoDubS_QWflu5xjRZt7nOmoWY56MbIM_shtbedSnlmzKzhC16vqqraobETkn3k4idgNpdtlbBgW85wvZmF9S5OrpL192SLaF1tErIHswrYuuzlFoBGOAqXQKiJ2czbPgu87Geh0d1_m_x6edF0i)
- [barcelona-life.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHqgATcS_gloHzc0-y1th--o9XT_uYtvM7Oimc3-TG-X4a6kLtqg8ihZGwA_RtwOltZEcCJG9knBRN6ZbLGRwRPFt5h451e8ljD3HcQsRP-xjtY8kfPXXH7GUdW6UepNJ8eDSIVag==)
- [barcelonacard.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGnPr6rCuRuvG_5IcMf3MmpBzsgNdObgPyAKQ6sNnZX7eB9HqfhMAO_oxDzKIpDJ3BFS5xz4rQoNO-WOPV16wcnyK3PwMbVkthZGqNAVaqRXShW6akZBRl6OzaItEt3bnCCy4mRljoLpvI-TdXU)
- [ticketshop.barcelona](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHStr_ZlIWdDrF1uc-3KY9Ub-_WRPhSEpBf0BheTX6Gq65WVCAH0kkC7qUPbmxWERF5A71oO9I7cH-M68bY70moS-RAIPtmVdhYksr_dDDgzNcPvMAVzcYxhm_0gDWh9ef9d1o-LR_g)
- [agoda.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQErJ9uv2p7iChDPMbBLgEwbOUQhcAWvnKGZDiXydIXoFL7FImgBF5mbK6wJe0ODH_wDd0by35fOJFdse2osWUVjMQU6NdWk0TlPiKApvRVoinMfiTdgaJg5JQm_yoYH8H95kpMJcCkETxOjuRLbAY2VekYSaxIN-aS5-dcL9K5kgaEkqvtzMYiDega4EmLkzX4zvqZy3ocymfWDgY8feA8i_mg=)
- [viator.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHjjSH9C99Hv0bXnQmg20RYVVYPGXZ5iGhRjmZz8vufOLGj1LkA6VT5XpSk9gAMWTZCsAqsbukaANVT9jPFgw0cE3bqA67xKocTRhDoidoM2jo28gRtZVGwJpxBAo_C74Wb0a7LISwi__owlOBG3TBEYuRfuY5jzQ6jepc6Thd6IJwxNLufkfAapKJ7m7bd6OMufpqPfmX3sJscOClsTYI9NiBXiph6EX_hDA==)
- [casabatllotickets.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFbfyNUF7-UuDNFeL8x3mahvUNOShCiqki4tHkxn_OSXp708vB2ewfrfArx3NVptJEDzpKqKrci3LGdeyGVLPn2arZdq6QOxV8Da7J_QAO-2-op4dqeRdK-3A==)
- [timeout.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIOjFNTfaW4v0BseWOaeuPrQqpvfhaXi4M99E15w7ZXLwo_E-K6ATCqpIHNcjNOnpNV3NYSo-una7kh1MjuLXILvf7hR_D8TNfS722DmelqVRWBFfr3e_cVbSgJm9M80KlEpN1u3pdug4y91Elg7J55c4zbpqaAVQ3xkX17J7tVpgRIol-7g==)
- [bcn.travel](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEvIzY5OF8y52cyjjJmMA7nzib3h5lKbwzHo_AxAL2ldJPakfz5x2uaypsKUlIw9SVogcElJHnMAX_RTVD6pZZVRgff6l1rNpN8oy4oFauIQm7h8Zb0MvMBUk8wYHjF2Esq8neTaPjlMQ==)
- [theworlds50best.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG-dKQtfGdHMQLe2fgbu_6lDG2PFJGOO1XTVrd9lRdtQXZp1fbVJVr5ZCELoxs4TCx10lhYj48m7p2cWR8zvRdgIO7C0Uol8k46-ZTSfcS0Ie6okVWb1f2jvWKqW4GbZa9axOvwBZo1G-1qrmLCvkcIK9nOxcE1ixBeTR1l)
- [restaurantecasarincon.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_Ghgrn0AVH4p6S5FsFa2u0Z9RcaMLoY0yVF3nwAurrCEJnEd8SdSJucuCMizuiA4yqqle4GlVRbFt6C_7hFsBgE9hpZ8DpUpK7KE_916_s-f5gTPUxHD2Ud6U5y-pPRGp5-EmFcyHXQnBwpNccPdLY2cQprAZjjOchBZnnc-FB1xqj3F9bKfpsXaVwvgDbbuw2QZ-ahPci1yuQ5OQ9uBzE4wM_exOtcJ4M9C22Uq7uVs=)
- [viator.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHGkQgYp00urRds6LklAOquNALCgf3IxCtwqTNXtKftiUiyol--Wgu79U5U1vBxVC0ac3f5mJ44jAqZ1UpiFeJqLUCSo-xb9Q8-0GcbmAxLDf3-0JKK5DoqLY9pqYtHEeuW5tNue0wnbVTi546SrrJob-c_t9o46dG_58PqM2n9NhFMooyeIWMxETDHOGOSJUR5wyTZwCo4ZsiWF0WQJLGWFMYEwhxiVl3q)
- [barcelona-tickets.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFHr4A6Jjo-nOPtXLIH7ovzhdEKrHkk2Hn_hKhyk3vA25nm222fi2_MsKDmGzsxnkcekxlOl_UkQIP85Bo9HV0Ze4UM_t102nhS06OUX9BQpm13FTeK9ZK3Bz-ed1sZXtax0Bn-VeFNDX6TFAiRpBV-6doqyTPbHkX65qT49kpTJkkkr4D4cWhSvt7d82JTpw==)
- [palaumusica.cat](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGH4w12sSHJMD0Qpc9quZNxucf6bff20hudVDgF1q1csS0XdCFd9PdM-By8B4pmW-fEI6_8gAD1zytqX1GdpYdwRbi20bi9FaNyDI1sv23qyRaYaeh7oXEs2PC4ysn5MtMyKmipFG9v10g6hmBE2XtHUxZHo1oxv2GeH5A=)
- [palaumusica.cat](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH4NE_Kvo2le_VtDT1_gAlTuakjCvoiyjLyKQnITpFUoIJKeATtFGwedsPo2D8dipTCh6U97BAJZ5s2uEbpUHWzuV-NcsucXlQCDR2aqYcxCbtY9EvepI-2l9v9Zqw9jnLXAhXjPEq0-8Yzi5OqvXXyOayet17jT-Wkv1O0)
- [palaumusica.cat](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHm7x_tZLTz5gZSfG_9-eeD_YSLHkUtzyQLLAx1KiAJCcmPnltCy3G_hJZ6tgVBP6irjBPc7Cup4eFtQeL_Gb8CN9rokO-iTt4AWIKqRaSVzvAU7Y0QvLa3sHcgWfoX3ko9ku-nsEKyz4meTFjvJqJAkJ34boA=)

--- Itinerary Agent Interaction Complete ---


When I run this exercise the final answer did not include the links, I tried to fix it with copilot (see notes below), but because it did not work, I fixed manually by adding this to the itinerary_agent instructions
"Don't forget to include the links in the answer as a Markdown hyperlink like ```[Item Name](URL)```."


Notes from copilot fix. The original goal was to ensure links were included in the response as in the video, but the Copilot fix, showed the links in the metadata which was not the saem.

Copilot answer

Links are missing because the agent's human-readable text is in event.content.parts[*].text, while the search tool's URLs are stored in the event's grounding metadata (e.g., event.grounding_metadata.grounding_chunks[*].web.uri). Your current code concatenates only part.text, so it never includes those URIs.
Why that happens (from your debug output)

The debug shows parts with only text (the itinerary markdown).
The grounding sources (titles + URIs) appear under grounding_metadata.grounding_chunks[].web.uri.
Since run_adk_agent_interaction only gathers part.text, it omits the grounding chunk URIs entirely.
What to change (safe, robust fix)

Convert the runner events to a list (so we can inspect them and iterate multiple times).
Build the response from parts (keep existing text).
Extract grounding chunks (title + uri) and append a “Sources” Markdown section with links formatted as [title](uri).
Drop-in code you can paste into the run_adk_agent_interaction function

Replace the existing event handling/response assembly with the block below. It handles text, html, common URL attributes (in parts), and the grounding metadata URIs:

```
# Run the agent and collect events into a list (so we can inspect them safely)
events = list(runner.run(user_id=user_id, session_id=session_id, new_message=content))

# (Optional) debug print of events/parts (keep or remove as needed)
for event in events:
    print("EVENT:", event)
    for i, part in enumerate(event.content.parts):
        print(" PART", i, "repr:", repr(part))
        for attr in ("text", "html", "url", "uri", "href", "link"):
            if hasattr(part, attr):
                print("  ", attr, "=", getattr(part, attr))

# Helper to get readable text for a Part
def _part_to_text(part):
    if getattr(part, "text", None):
        return part.text
    if getattr(part, "html", None):
        return part.html
    for attr in ("url", "uri", "href", "link"):
        val = getattr(part, attr, None)
        if val:
            return f"[{val}]({val})"
    return str(part)

# Build final response and include grounding links if available
for event in events:
    if event.is_final_response():
        # Combine the parts' text/HTML
        response = "".join(_part_to_text(p) for p in event.content.parts)

        # Try to append grounding sources (if present)
        sources = []
        gm = getattr(event, "grounding_metadata", None)
        if gm and getattr(gm, "grounding_chunks", None):
            for chunk in gm.grounding_chunks:
                web = getattr(chunk, "web", None)
                if web:
                    title = getattr(web, "title", None) or getattr(web, "uri", None)
                    uri = getattr(web, "uri", None)
                    if uri:
                        # Use the title if present, otherwise display the URL as the label
                        label = title if title else uri
                        sources.append(f"- [{label}]({uri})")

        if sources:
            response += "\n\n**Sources:**\n" + "\n".join(sources)

        return response

return "No final response from agent."
```

How to apply & test

Option A (recommended): I can patch 02_03.ipynb and replace the event-processing code in the modular function with the snippet above, then you re-run the itinerary cell. Tell me to apply the patch and I’ll make the edit.
B (manual): open the cell that defines run_adk_agent_interaction, replace the event-processing block with the snippet, and run the itinerary cell again.
Notes and small caveats

The URIs in your grounding metadata are redirect URLs (Vertex AI grounding redirect links). They will work as links but may not show the final host in the label unless you use the title from the grounding chunk (we use that when available).
If you want the links inserted inline near the exact place they were used in the textual itinerary (rather than as a “Sources” list), we can implement a more advanced step that maps grounding support segments to locations in the text and inserts hyperlinks in-place. That’s slightly more work but doable.
Do you want me to:

Patch 02_03.ipynb now with this change and remove the debug prints, or
Apply the simpler fix and keep debug prints for one more run so you can confirm the exact mapping?

In [15]:
# Congratulations 🎉 You’ve now successfully built and run two different LLM agents in ADK:
# A Packing List Agent for smart, personalized packing suggestions.
# An Itinerary Agent that effectively uses a google_search tool to plan an amazing trip with live, up-to-date recommendations.
# This demonstrates the power of ADK in combining LLM reasoning with external tools to create truly intelligent and practical applications. 
# This is the foundation for building even richer, multi-agent travel assistants and other sophisticated AI solutions with Google ADK.